# preprocess data
split the eeg into small chunks

In [4]:
from tqdm.notebook import tqdm
import pandas as pd
import torch as t
import matplotlib.pyplot as plt

In [5]:
OUT_DIR = './spectrograms/'
BASE_PATH = './hms-harmful-brain-activity-classification/'
DATA_PATH = './hms-harmful-brain-activity-classification/train_spectrograms/'

In [13]:
def preprocess(df):
    duration = 600
    prefixes = ["LL","RL","RP","LP"]
    for spectrogram_id, group in tqdm(df.groupby('spectrogram_id')):
        parquet_file = f'{DATA_PATH}{spectrogram_id}.parquet'
        raw_spectrogram = pd.read_parquet(parquet_file)
        for _, row in group.iterrows():
            spectrogram_sub_id = row['spectrogram_sub_id']
            offset = int(row['spectrogram_label_offset_seconds'])
            spectrogram = raw_spectrogram.loc[(raw_spectrogram.time >= offset) & (raw_spectrogram.time < offset + duration)]
            # offset = int(row['eeg_label_offset_seconds'] * sample_rate)
            # eeg = raw_eeg.iloc[offset:offset + duration]
            spectrogram = spectrogram.ffill(axis=0).fillna(0)
            traces = []
            for prefix in prefixes:
                df = spectrogram.filter(regex=f'^{prefix}', axis=1)
                traces.append(t.tensor(df.values))
            data = t.stack(traces)
            # print(f'{data.shape=}')
            # for trace_id in range(4):
            #     plt.imshow(data[trace_id].permute(1, 0))
            #     plt.show()
            t.save(data, f'{OUT_DIR}{spectrogram_id}_{spectrogram_sub_id}.pt')

df = pd.read_csv(f'{BASE_PATH}train.csv')
preprocess(df)

  0%|          | 0/11138 [00:00<?, ?it/s]

./spectrograms/353733_0.pt
./spectrograms/353733_1.pt
./spectrograms/353733_2.pt
./spectrograms/353733_3.pt
./spectrograms/353733_4.pt
./spectrograms/353733_5.pt
./spectrograms/353733_6.pt
./spectrograms/353733_7.pt
./spectrograms/353733_8.pt
